In [ ]:
!pip install scikit-learn pandas

In [ ]:
import pandas as pd

# Load the training data
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

# Display the first few rows to understand its structure
train_df.head()


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [ ]:
train_df.isnull().sum()

,0
PassengerId,0
Survived,0
Pclass,0
Name,0
Sex,0
Age,0
SibSp,0
Parch,0
Ticket,0
Fare,0


In [ ]:
from sklearn.impute import SimpleImputer
import pandas as pd

# Convert the column to string type (optional but good to ensure consistency)
train_df['Cabin'] = train_df['Cabin'].astype(str)

# Define the imputer
imputer = SimpleImputer(strategy='most_frequent')

# Fit and transform the data
train_df[['Cabin']] = imputer.fit_transform(train_df[['Cabin']])

# Check for missing values after imputation
print(train_df.isnull().sum())



PassengerId    0
Survived       0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Cabin          0
Embarked       0
dtype: int64


<ipython-input-21-eedfd818e668>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df['Cabin'] = train_df['Cabin'].astype(str)
<ipython-input-21-eedfd818e668>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df[['Cabin']] = imputer.fit_transform(train_df[['Cabin']])


In [ ]:
from sklearn.preprocessing import OneHotEncoder

# One-hot encode the 'Embarked' column
# Changed 'handle_unknown' to 'ignore'
one_hot_encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
embarked_encoded = one_hot_encoder.fit_transform(train_df[['Cabin']])

# Create a new DataFrame for the encoded columns
encoded_columns = one_hot_encoder.categories_[0]
Cabin_df = pd.DataFrame(embarked_encoded, columns=encoded_columns)

# Drop the original 'Embarked' column and concatenate with encoded columns
# You are dropping Embarked but encoding Cabin
# I am assuming this is what you intended.
train_df = train_df.drop('Embarked', axis=1)
train_df = pd.concat([train_df, Cabin_df], axis=1)

# Display the first few rows to see the result
train_df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,...,F E69,F G63,F G73,F2,F33,F38,F4,G6,T,nan
0,1.0,0.0,3.0,"Braund, Mr. Owen Harris",male,22.0,1.0,0.0,A/5 21171,7.2500,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,2.0,1.0,1.0,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1.0,0.0,PC 17599,71.2833,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3.0,1.0,3.0,"Heikkinen, Miss. Laina",female,26.0,0.0,0.0,STON/O2. 3101282,7.9250,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,4.0,1.0,1.0,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1.0,0.0,113803,53.1000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5.0,0.0,3.0,"Allen, Mr. William Henry",male,35.0,0.0,0.0,373450,8.0500,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier



In [ ]:
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), ['Age']),  # Scale 'Age' column
        ('cat', OneHotEncoder(handle_unknown='ignore'), ['Embarked'])  # One-hot encode 'Embarked' column
    ])

# Model (e.g., RandomForestClassifier)
model = RandomForestClassifier()


In [ ]:
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', model)
])


In [ ]:
# prompt: check the column Survived  is there or not

if 'Survived' in train_df.columns:
  print("Survived column is present in the DataFrame.")
else:
  print("Survived column is not present in the DataFrame.")

Survived column is present in the DataFrame.


In [ ]:

imputer = SimpleImputer(strategy='most_frequent')
train_df['Survived'] = imputer.fit_transform(train_df[['Survived']]).ravel()


In [ ]:
pipeline.fit(train_df.drop('Survived', axis=1), train_df['Survived'])


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num', StandardScaler(),
                                                  ['Age']),
                                                 ('cat',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse_output=False),
                                                  ['Cabin'])])),
                ('model', RandomForestClassifier())])

In [ ]:
predictions = pipeline.predict(test_df)
predictions


array([1., 0., 0., 1., 1., 1., 1., 1., 1., 1., 0., 0., 1., 0., 0., 1., 1.,
       1., 1., 0., 0., 1., 0., 1., 1., 0., 1., 1., 1., 0., 0., 1., 1., 0.,
       1., 1., 0., 1., 1., 0., 1., 0., 1., 1., 1., 1., 0., 0., 1., 1., 1.,
       1., 1., 1., 0., 1., 1., 0., 0., 1., 1., 1., 1., 1., 1., 0., 1., 0.,
       1., 0., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 0., 0., 0., 0.,
       0., 1., 1., 0., 0., 1., 0., 0., 0., 1., 1., 0., 1., 1., 1., 1., 1.,
       0., 1., 1., 1., 1., 0., 0., 1., 1., 0., 1., 1., 0., 1., 0., 0., 1.,
       1., 1., 0., 1., 1., 0., 1., 1., 0., 1., 1., 1., 1., 0., 0., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 0., 1., 1., 0., 0.,
       1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 0., 1., 1., 1., 1., 0., 1.,
       0., 1., 1., 0., 1., 1., 1., 1., 1., 0., 1., 1., 0., 0., 0., 1., 1.,
       1., 0., 1., 1., 0., 1., 0., 1., 1., 1., 1., 1., 0., 0., 1., 1., 1.,
       1., 0., 1., 1., 1., 1., 1., 0., 1., 0., 1., 1., 0., 0., 1., 0., 1.,
       1., 1., 1., 0., 0.

In [ ]:
from sklearn.model_selection import cross_val_score

scores = cross_val_score(pipeline, train_df.drop('Survived', axis=1), train_df['Survived'], cv=5)
print(f"Cross-validation scores: {scores}")
print(f"Mean accuracy: {scores.mean()} ± {scores.std()}")


Cross-validation scores: [0.68156425 0.65168539 0.69662921 0.71348315 0.73033708]
Mean accuracy: 0.694739815454146 ± 0.027016018657821165


In [ ]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

param_grid = {
    'model__n_estimators': [50, 100, 150],
    'model__max_depth': [3, 5, 7, None],
    'model__min_samples_split': [2, 5, 10]
}

grid_search = GridSearchCV(pipeline, param_grid, cv=5, n_jobs=-1)
grid_search.fit(train_df.drop('Survived', axis=1), train_df['Survived'])

print(f"Best parameters: {grid_search.best_params_}")
print(f"Best score: {grid_search.best_score_}")


Best parameters: {'model__max_depth': 7, 'model__min_samples_split': 10, 'model__n_estimators': 100}
Best score: 0.698135710250455
